In [10]:
import os

In [11]:
%pwd

'/home/user/Documents/ML DL PROJECTS/ChickenDiseaseClassificationDLProject'

In [12]:
%cd /home/user/Documents/ML DL PROJECTS/ChickenDiseaseClassificationDLProject/

/home/user/Documents/ML DL PROJECTS/ChickenDiseaseClassificationDLProject


In [38]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [39]:
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import read_yaml,create_directories

In [40]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])
    def getDataIngestionConfig(self)->DataIngestionConfig:
        config  = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(root_dir = config.root_dir, source_url = config.source_url, local_data_file = config.local_data_file, unzip_dir = config.unzip_dir)
        return data_ingestion_config

In [41]:
import os
import urllib.request as request
import zipfile
from ChickenDiseaseClassification import logger
from ChickenDiseaseClassification.utils.common import get_size

In [45]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url=self.config.source_url, filename=self.config.local_data_file)
            logger.info(f"{filename} downloaded successfully with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [47]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.getDataIngestionConfig()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 



[2023-10-07 09:06:10,512: INFO: common yaml file: config/config.yaml loaded succuessfully]
[2023-10-07 09:06:10,516: INFO: common yaml file: params.yaml loaded succuessfully]
[2023-10-07 09:06:10,518: INFO: common directory at aritfacts created succuessfully]
[2023-10-07 09:06:10,521: INFO: common directory at artifacts/data_ingestion created succuessfully]
[2023-10-07 09:06:55,106: INFO: 1563732529 artifacts/data_ingestion/data.zip downloaded successfully with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6650:D7EB6:1343F5:160D85:6520D0C3
Accept-Ranges: bytes
Date: Sat, 07 Oct 2023 03:35:08 GMT
Via: 1.1 varnish
X-Ser